In [82]:
DATABASE_HOST="172.20.0.16"
# DATABASE_HOST="172.20.0.3"
DATABASE_DATABASE="fabricexplorer"
DATABASE_USERNAME="hppoc"
DATABASE_PASSWORD="password"
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import bamboolib as bam
# import hiplot as hip
import matplotlib.pyplot as ptlib
import plotly.express as px
import json

# to get the ips of docker containers
# docker ps -q | xargs -n 1 docker inspect --format '{{ .Name }} {{range .NetworkSettings.Networks}} {{.IPAddress}}{{end}}' | sed 's#^/##';
# list all user defined tables and schemas
# "SELECT * FROM pg_catalog.pg_tables WHERE schemaname != 'information_schema' AND schemaname != 'pg_catalog';"

#initializing resources
engine = create_engine('postgresql://'+DATABASE_USERNAME+':'+DATABASE_PASSWORD+'@'+DATABASE_HOST+':5432/'+DATABASE_DATABASE)

txQuery = "SELECT * FROM transactions"
txDf = pd.read_sql(txQuery,con=engine)
# txDf.to_csv('backup.csv')
txDf=txDf[txDf['read_set'].isna()==False]
txDf=txDf[txDf['chaincodename'].str.contains('_lifecycle')==False]
txDf=txDf[txDf['type'].str.contains('CONFIG')==False]
txDf=txDf.drop(columns=["txhash","chaincode_id","endorser_msp_id","type","createdt","channel_genesis_hash","envelope_signature","creator_id_bytes", "creator_nonce", "payload_extension","tx_response","payload_proposal_hash","endorser_id_bytes","endorser_signature", "network_name"])
txDf

        id  blockid chaincodename  status    creator_msp_id  \
0        6        6        simple   200.0  org0-example-com   
10      16       10      samplecc   200.0  org0-example-com   
11      17       11      samplecc   200.0  org0-example-com   
12      18       11      samplecc   200.0  org0-example-com   
13      19       11      samplecc   200.0  org0-example-com   
...    ...      ...           ...     ...               ...   
9898  9904     1001      samplecc   200.0  org0-example-com   
9899  9905     1001      samplecc   200.0  org0-example-com   
9900  9906     1001      samplecc   200.0  org0-example-com   
9901  9907     1002      samplecc   200.0  org0-example-com   
9902  9908     1002      samplecc   200.0  org0-example-com   

                                               read_set  \
0     [{'chaincode': '_lifecycle', 'set': [{'key': '...   
10    [{'chaincode': '_lifecycle', 'set': [{'key': '...   
11    [{'chaincode': '_lifecycle', 'set': [{'key': '...   
12    [

In [83]:
read_set=txDf["read_set"]
write_set=txDf["write_set"]
for item in read_set:
    item.pop(0)
for item in write_set:
    item.pop(0)

In [84]:
txDf['read_set']=read_set
txDf['write_set']=write_set
txDf = txDf.explode('read_set')
txDf = txDf.explode('write_set')
txDf = txDf.loc[(txDf['read_set'].astype('string').str.contains('version', case=False, regex=False, na=False))]
txDf

        id  blockid chaincodename  status    creator_msp_id  \
11      17       11      samplecc   200.0  org0-example-com   
12      18       11      samplecc   200.0  org0-example-com   
13      19       11      samplecc   200.0  org0-example-com   
14      20       11      samplecc   200.0  org0-example-com   
15      21       11      samplecc   200.0  org0-example-com   
...    ...      ...           ...     ...               ...   
9898  9904     1001      samplecc   200.0  org0-example-com   
9899  9905     1001      samplecc   200.0  org0-example-com   
9900  9906     1001      samplecc   200.0  org0-example-com   
9901  9907     1002      samplecc   200.0  org0-example-com   
9902  9908     1002      samplecc   200.0  org0-example-com   

                                               read_set  \
11    {'chaincode': 'samplecc', 'set': [{'key': ' 􏿿i...   
12    {'chaincode': 'samplecc', 'set': [{'key': ' 􏿿i...   
13    {'chaincode': 'samplecc', 'set': [{'key': ' 􏿿i...   
14    {

In [85]:
txDf['r_chaincode'] =txDf['read_set'].apply(lambda x: x['chaincode'])
txDf['rs'] =txDf['read_set'].apply(lambda x: x['set'])
txDf = txDf.drop(columns=['chaincodename','read_set'])
txDf['w_chaincode'] =txDf['write_set'].apply(lambda x: x['chaincode'])
txDf['ws'] =txDf['write_set'].apply(lambda x: x['set'])
txDf = txDf.drop(columns=['write_set'])
txDf

        id  blockid  status    creator_msp_id validation_code  \
11      17       11   200.0  org0-example-com           VALID   
12      18       11   200.0  org0-example-com           VALID   
13      19       11   200.0  org0-example-com           VALID   
14      20       11   200.0  org0-example-com           VALID   
15      21       11   200.0  org0-example-com           VALID   
...    ...      ...     ...               ...             ...   
9898  9904     1001   200.0  org0-example-com           VALID   
9899  9905     1001   200.0  org0-example-com           VALID   
9900  9906     1001   200.0  org0-example-com           VALID   
9901  9907     1002   200.0  org0-example-com           VALID   
9902  9908     1002   200.0  org0-example-com           VALID   

                               chaincode_proposal_input r_chaincode  \
11    690760065,707574,30030034353335323738,30034353...    samplecc   
12    690760065,707574,310300383630373537,3103836303...    samplecc   
13    

In [86]:
def classifyWrite(ws):
    if str(ws)=='nan':
        return np.nan
    elif ws['is_delete']:
        return 'DELETE'
    else:
        return 'WRITE'
txDfr = txDf.drop(columns=['w_chaincode','ws']).explode('rs')
txDfr = txDfr.rename(columns={'r_chaincode': 'chaincode','rs':'key'})
txDfr['access_type'] = txDfr['key'].apply(lambda x: 'READ')
txDfr['version_block'] = txDfr['key'].apply(lambda x: np.nan if str(x) == 'nan' else x['version']['block_num'])
txDfr['version_tx'] = txDfr['key'].apply(lambda x: np.nan if str(x) == 'nan' else x['version']['tx_num'])
txDfw = txDf.drop(columns=['r_chaincode','rs']).explode('ws')
txDfw = txDfw.rename(columns={'w_chaincode': 'chaincode','ws':'key'})
txDfw['access_type'] = txDfw['key'].apply(lambda x: classifyWrite(x))
txDf = txDfr.append(txDfw).reset_index(drop=True).rename(columns={'id':'txid'})
txDf['key']=txDf['key'].apply(lambda x:np.nan if str(x) == 'nan' else x['key'])

In [87]:
txDf

       txid  blockid  status    creator_msp_id validation_code  \
0        17       11   200.0  org0-example-com           VALID   
1        18       11   200.0  org0-example-com           VALID   
2        19       11   200.0  org0-example-com           VALID   
3        20       11   200.0  org0-example-com           VALID   
4        21       11   200.0  org0-example-com           VALID   
...     ...      ...     ...               ...             ...   
19779  9904     1001   200.0  org0-example-com           VALID   
19780  9905     1001   200.0  org0-example-com           VALID   
19781  9906     1001   200.0  org0-example-com           VALID   
19782  9907     1002   200.0  org0-example-com           VALID   
19783  9908     1002   200.0  org0-example-com           VALID   

                                chaincode_proposal_input chaincode  \
0      690760065,707574,30030034353335323738,30034353...  samplecc   
1      690760065,707574,310300383630373537,3103836303...  samplecc 

In [88]:
query = "SELECT * FROM pg_catalog.pg_tables WHERE schemaname != 'information_schema' AND schemaname != 'pg_catalog';"
# query = "select * from peer"
idkdf = pd.read_sql(query,con=engine)
idkdf

  schemaname           tablename tableowner tablespace  hasindexes  hasrules  \
0     public              blocks      hppoc       None        True     False   
1     public          chaincodes      hppoc       None        True     False   
2     public  peer_ref_chaincode      hppoc       None        True     False   
3     public             channel      hppoc       None        True     False   
4     public                peer      hppoc       None        True     False   
5     public    peer_ref_channel      hppoc       None        True     False   
6     public             orderer      hppoc       None        True     False   
7     public        transactions      hppoc       None        True     False   
8     public               users      hppoc       None        True     False   
9     public          write_lock      hppoc       None        True     False   

   hastriggers  rowsecurity  
0        False        False  
1        False        False  
2        False        False  